In [143]:
import sys
import pprint
import numpy as np
import tensorflow as tf
from qkeras.utils import _add_supported_quantized_objects
co = {}
_add_supported_quantized_objects(co)
from qkeras import print_qstats

# for automatic quantization
import pprint
from qkeras.autoqkeras import *
from qkeras import *
from qkeras.utils import model_quantize
from qkeras.qtools import run_qtools, interface
from qkeras.qtools import settings as qtools_settings
from qkeras import quantized_bits
from qkeras import QDense, QActivation
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from qkeras import quantized_bits


In [144]:
with open('run_config.json', 'r') as f:
    run_config = json.load(f)
aq = autoqkeras.forgiving_metrics.ForgivingFactorBits(8, 8, 2, config=run_config['quantization_config'])

In [145]:
def build_qe(image_size=16):
#     inputs = tf.keras.Input((16),name="Input")
#     x = QDense(32, kernel_quantizer = quantized_bits(4,0,1), use_bias=False, name="qdense_1")(inputs)
#     x = QBatchNormalization()(x)
#     x = QActivation('quantized_relu(4,2)',name="qact_1")(x)
    
#     x = QDense(16, kernel_quantizer = 'ternary', use_bias=False, name="qdense_2")(x)
#     x = QBatchNormalization()(x)
#     x = QActivation('quantized_relu(3,1)',name="qact_2")(x)
    
#     x = QDense(16, kernel_quantizer = quantized_bits(2,1,1), use_bias=False, name="qdense_3")(x)
#     x = QBatchNormalization()(x)
#     x = QActivation('quantized_relu(4,2)',name="qact_3")(x)
    
#     x = QDense(5, kernel_quantizer = 'stochastic_binary', use_bias=False, name="qdense_nclasses")(x)
#     predictions = tf.keras.layers.Activation('softmax',name="softmax")(x)
#     model = tf.keras.Model(inputs, predictions,name='baseline')
    model = tf.keras.models.load_model('QE.h5',custom_objects={'PruneLowMagnitude': pruning_wrapper.PruneLowMagnitude,'QDense': QDense, 'QConv2D': QConv2D, 'Clip': Clip, 'QActivation': QActivation})
    
    return model

def build_qb(image_size=16):
#     inputs = tf.keras.Input((16),name="Input")
#     x = QDense(32, kernel_quantizer = quantized_bits(4,0,1), use_bias=False, name="qdense_1")(inputs)
#     x = QBatchNormalization()(x)
#     x = QActivation('quantized_relu(4,2)',name="qact_1")(x)
    
#     x = QDense(16, kernel_quantizer = 'stochastic_binary', use_bias=False, name="qdense_2")(x)
#     x = QBatchNormalization()(x)
#     x = QActivation('quantized_relu(3,1)',name="qact_2")(x)
    
#     x = QDense(16, kernel_quantizer = 'stochastic_binary', use_bias=False, name="qdense_3")(x)
#     x = QBatchNormalization()(x)
#     x = QActivation('quantized_relu(4,2)',name="qact_3")(x)
    
#     x = QDense(5, kernel_quantizer = 'stochastic_binary', use_bias=False, name="qdense_nclasses")(x)
#     predictions = tf.keras.layers.Activation('softmax',name="softmax")(x)
#    model = tf.keras.Model(inputs, predictions,name='baseline')
    model = tf.keras.models.load_model('QB.h5',custom_objects={'PruneLowMagnitude': pruning_wrapper.PruneLowMagnitude,'QDense': QDense, 'QConv2D': QConv2D, 'Clip': Clip, 'QActivation': QActivation})
    
    return model

def build_q6(image_size=16):
#     inputs = tf.keras.Input((16),name="Input")
#     x = QDense(64, kernel_quantizer = quantized_bits(6,0,1), bias_quantizer = quantized_bits(6,0,1))(inputs)
#     x = QActivation('quantized_relu(6,0)',name="qact_1")(x)
#     x = QDense(32, kernel_quantizer = quantized_bits(6,0,1), bias_quantizer = quantized_bits(6,0,1))(x)
#     x = QActivation('quantized_relu(6,0)',name="qact_2")(x)
#     x = QDense(32, kernel_quantizer = quantized_bits(6,0,1), bias_quantizer = quantized_bits(6,0,1))(x)
#     x = QActivation('quantized_relu(6,0)',name="qact_3")(x)
#     x = QDense(5, kernel_quantizer = quantized_bits(6,0,1), bias_quantizer = quantized_bits(6,0,1))(x)
#     predictions = tf.keras.layers.Activation('softmax',name="softmax")(x)
#     model = tf.keras.Model(inputs, predictions,name='baseline')
    model = tf.keras.models.load_model('Q6.h5',custom_objects={'PruneLowMagnitude': pruning_wrapper.PruneLowMagnitude,'QDense': QDense, 'QConv2D': QConv2D, 'Clip': Clip, 'QActivation': QActivation})
    
    return model

def build_bf(image_size=16):
    inputs = tf.keras.Input((16),name="Input")
    x = QDense(64, kernel_quantizer = quantized_bits(15,5,1), bias_quantizer = quantized_bits(15,5,1), name="fc1")(inputs)
    x = QActivation('quantized_relu(15,5)',name="relu1")(x)
    x = QDense(32, kernel_quantizer = quantized_bits(15,5,1), bias_quantizer = quantized_bits(15,5,1), name="fc2")(x)
    x = QActivation('quantized_relu(15,5)',name="relu2")(x)
    x = QDense(32, kernel_quantizer = quantized_bits(15,5,1), bias_quantizer = quantized_bits(15,5,1), name="fc3")(x)
    x = QActivation('quantized_relu(15,5)',name="relu3")(x)
    x = QDense(5, kernel_quantizer = quantized_bits(15,5,1), bias_quantizer = quantized_bits(15,5,1), name="output")(x)
    predictions = tf.keras.layers.Activation('softmax',name="softmax")(x)
    model = tf.keras.Model(inputs, predictions,name='baseline')
    return model

In [154]:
def getEstimates(model):
    print('For model {}'.format(model.name))
    q = run_qtools.QTools(model, 
                          process="horowitz", 
                          source_quantizers=[quantized_bits(15, 5, 1)], 
                          is_inference=True, 
                          weights_path=None,
                          keras_quantizer="fp16",
                          keras_accumulator="fp16", 
                          for_reference=False)
    total_bitsize = 0
    total_parbitsize = 0
    total_actbitsize = 0
    perLayerBitSizes_param = {}
    perLayerBitSizes_act = {}
    for layer in model.layers:
        if layer.name.find('nput')!=-1:
            continue
        layer_name = layer.__class__.__name__
        parameters = aq._param_size(layer)
        activations = aq._act_size(layer)
        pprint.pprint('Layer = {} Parameter bitsize = {}'.format(layer.name,parameters))
        pprint.pprint('Layer = {} Activation bitsize = {}'.format(layer.name,activations))
        total_parbitsize  += parameters
        total_actbitsize  += activations
        perLayerBitSizes_param[layer.name] = parameters
        perLayerBitSizes_act[layer.name]   = activations
        total_bitsize += parameters+activations
    print('\n Total bit size = {} \n'.format(total_bitsize))
    print('Used for weights = {} \n'.format(total_parbitsize))
    print('Used for activations = {} \n'.format(total_actbitsize))
    # caculate energy of the derived data type map.
    energy_dict = q.pe(
        # whether to store parameters in dram, sram, or fixed
        weights_on_memory="fixed",
        # store activations in dram or sram
        activations_on_memory="fixed",
        # minimum sram size in number of bits. Let's assume a 16MB SRAM.
        min_sram_size=8*16*1024*1024,
        # whether load data from dram to sram (consider sram as a cache
        # for dram. If false, we will assume data will be already in SRAM
        rd_wr_on_io=False)
    # get stats of energy distribution in each layer
    energy_profile = q.extract_energy_profile(
        qtools_settings.cfg.include_energy, energy_dict)
    # extract sum of energy of each layer according to the rule specified in
    # qtools_settings.cfg.include_energy
    total_energy = q.extract_energy_sum(
        qtools_settings.cfg.include_energy, energy_dict)
    perLayerEnergy = {}
    for key in energy_profile.keys():
        pprint.pprint('Layer = {} Energy = {} pJ'.format(key, energy_profile[key]['energy']['op_cost'] ))
        perLayerEnergy[key] = energy_profile[key]['energy']['op_cost']
    print()
    print("\n Total energy: {:.1f} pJ \n".format(total_energy))
    return total_bitsize, perLayerBitSizes_param, perLayerBitSizes_act, total_energy, perLayerEnergy
    

In [155]:
total_bitsize       = {}
perLayerBitSize_par = {} 
perLayerBitSize_act = {} 
total_energy        = {} 
perLayerEnergy      = {}

In [156]:
model = build_bf()
total_bitsize['BF'], perLayerBitSize_par['BF'], perLayerBitSize_act['BF'], total_energy['BF'], perLayerEnergy['BF'] = getEstimates(model)

For model baseline
'Layer = fc1 Parameter bitsize = 16320'
'Layer = fc1 Activation bitsize = 0'
'Layer = relu1 Parameter bitsize = 0'
'Layer = relu1 Activation bitsize = 960'
'Layer = fc2 Parameter bitsize = 31200'
'Layer = fc2 Activation bitsize = 0'
'Layer = relu2 Parameter bitsize = 0'
'Layer = relu2 Activation bitsize = 480'
'Layer = fc3 Parameter bitsize = 15840'
'Layer = fc3 Activation bitsize = 0'
'Layer = relu3 Parameter bitsize = 0'
'Layer = relu3 Activation bitsize = 480'
'Layer = output Parameter bitsize = 2475'
'Layer = output Activation bitsize = 0'
'Layer = softmax Parameter bitsize = 0'
'Layer = softmax Activation bitsize = 40'

 Total bit size = 67795 

Used for weights = 65835 

Used for activations = 1960 

'Layer = fc1 Energy = 818.0 pJ'
'Layer = relu1 Energy = 0.0 pJ'
'Layer = fc2 Energy = 1648.8 pJ'
'Layer = relu2 Energy = 0.0 pJ'
'Layer = fc3 Energy = 821.2 pJ'
'Layer = relu3 Energy = 0.0 pJ'
'Layer = output Energy = 128.31 pJ'
'Layer = softmax Energy = 0.0 pJ'




In [157]:
model = build_q6()
total_bitsize['Q6'], perLayerBitSize_par['Q6'], perLayerBitSize_act['Q6'], total_energy['Q6'], perLayerEnergy['Q6'] = getEstimates(model)

For model model
'Layer = fc1 Parameter bitsize = 6528'
'Layer = fc1 Activation bitsize = 0'
'Layer = relu1 Parameter bitsize = 0'
'Layer = relu1 Activation bitsize = 384'
'Layer = fc2 Parameter bitsize = 12480'
'Layer = fc2 Activation bitsize = 0'
'Layer = relu2 Parameter bitsize = 0'
'Layer = relu2 Activation bitsize = 192'
'Layer = fc3 Parameter bitsize = 6336'
'Layer = fc3 Activation bitsize = 0'
'Layer = relu3 Parameter bitsize = 0'
'Layer = relu3 Activation bitsize = 192'
'Layer = output Parameter bitsize = 990'
'Layer = output Activation bitsize = 0'
'Layer = softmax Parameter bitsize = 0'
'Layer = softmax Activation bitsize = 40'

 Total bit size = 27142 

Used for weights = 26334 

Used for activations = 808 

'Layer = fc1 Energy = 369.32 pJ'
'Layer = relu1 Energy = 0.0 pJ'
'Layer = fc2 Energy = 355.2 pJ'
'Layer = relu2 Energy = 0.0 pJ'
'Layer = fc3 Energy = 174.4 pJ'
'Layer = relu3 Energy = 0.0 pJ'
'Layer = output Energy = 27.25 pJ'
'Layer = softmax Energy = 0.0 pJ'


 Total e

In [158]:
model = build_qb()
for layer in model.layers:
    layer._name = layer.name.replace('_relu','')
total_bitsize['QB'], perLayerBitSize_par['QB'], perLayerBitSize_act['QB'], total_energy['QB'], perLayerEnergy['QB'] = getEstimates(model)

For model model
'Layer = fc1 Parameter bitsize = 2048'
'Layer = fc1 Activation bitsize = 0'
'Layer = bn1 Parameter bitsize = 512'
'Layer = bn1 Activation bitsize = 0'
'Layer = relu1 Parameter bitsize = 0'
'Layer = relu1 Activation bitsize = 128'
'Layer = fc2 Parameter bitsize = 512'
'Layer = fc2 Activation bitsize = 0'
'Layer = bn2 Parameter bitsize = 256'
'Layer = bn2 Activation bitsize = 0'
'Layer = relu2 Parameter bitsize = 0'
'Layer = relu2 Activation bitsize = 64'
'Layer = fc3 Parameter bitsize = 512'
'Layer = fc3 Activation bitsize = 0'
'Layer = bn3 Parameter bitsize = 256'
'Layer = bn3 Activation bitsize = 0'
'Layer = relu3 Parameter bitsize = 0'
'Layer = relu3 Activation bitsize = 48'
'Layer = output_softmax Parameter bitsize = 120'
'Layer = output_softmax Activation bitsize = 0'
'Layer = softmax Parameter bitsize = 0'
'Layer = softmax Activation bitsize = 40'

 Total bit size = 4496 

Used for weights = 4216 

Used for activations = 280 

'Layer = fc1 Energy = 134.53 pJ'
'Laye

In [159]:
model = build_qe()
for layer in model.layers:
    layer._name = layer.name.replace('_relu','')
total_bitsize['QE'], perLayerBitSize_par['QE'], perLayerBitSize_act['QE'], total_energy['QE'], perLayerEnergy['QE'] = getEstimates(model)

For model model
'Layer = fc1 Parameter bitsize = 2048'
'Layer = fc1 Activation bitsize = 0'
'Layer = bn1 Parameter bitsize = 512'
'Layer = bn1 Activation bitsize = 0'
'Layer = relu1 Parameter bitsize = 0'
'Layer = relu1 Activation bitsize = 128'
'Layer = fc2 Parameter bitsize = 1024'
'Layer = fc2 Activation bitsize = 0'
'Layer = bn2 Parameter bitsize = 256'
'Layer = bn2 Activation bitsize = 0'
'Layer = relu2 Parameter bitsize = 0'
'Layer = relu2 Activation bitsize = 48'
'Layer = fc3 Parameter bitsize = 512'
'Layer = fc3 Activation bitsize = 0'
'Layer = bn3 Parameter bitsize = 256'
'Layer = bn3 Activation bitsize = 0'
'Layer = relu3 Parameter bitsize = 0'
'Layer = relu3 Activation bitsize = 64'
'Layer = output_softmax Parameter bitsize = 120'
'Layer = output_softmax Activation bitsize = 0'
'Layer = softmax Parameter bitsize = 0'
'Layer = softmax Activation bitsize = 40'

 Total bit size = 5008 

Used for weights = 4728 

Used for activations = 280 

'Layer = fc1 Energy = 134.53 pJ'
'Lay

In [160]:
#Let's also look at the relative energies and bit sizes per layer
models = ['BF','Q6','QE','QB']
pprint.pprint('{}      {}   {} {} {} {} {}'.format('Model', 'Bits', 'Energy [pJ]', 'Bits/BF', 'Bits/Q6', 'Energy/BF', 'Energy/Q6'))
for i,model in enumerate(models):
    pprint.pprint('{} {:.0f}     {:.0f}       {:.2f}     {:.2f}     {:.2f}     {:.2f}'.format('{} QTools'.format(model), total_bitsize[model], total_energy[model], total_bitsize[model]/total_bitsize['BF'], total_bitsize[model]/total_bitsize['Q6'], total_energy[model]/total_energy['BF'], total_energy[model]/total_energy['Q6']))
    pprint.pprint('{} {}         {}          {}          {}      {:.2f}   {:.2f}'.format('{} Vivado'.format(model), '-', '-', '-', '-', vivadoEBF[i], vivadoEQ6[i])) 
    pprint.pprint('')

'Model      Bits   Energy [pJ] Bits/BF Bits/Q6 Energy/BF Energy/Q6'
'BF QTools 67795     3439       1.00     2.50     1.00     3.62'
'BF Vivado -         -          -          -      1.00   5.81'
''
'Q6 QTools 27142     949       0.40     1.00     0.28     1.00'
'Q6 Vivado -         -          -          -      0.17   1.00'
''
'QE QTools 5008     193       0.07     0.18     0.06     0.20'
'QE Vivado -         -          -          -      0.05   0.30'
''
'QB QTools 4496     181       0.07     0.17     0.05     0.19'
'QB Vivado -         -          -          -      0.04   0.25'
''


In [161]:
#Let's look at the relative energies and bit sizes per layer
models = ['BF','Q6','QE','QB']

for i,model in enumerate(models):
    print('\n For model {}'.format(model))
    perLayerBrelQ6 = {}
    perLayerBrelBF = {}
    perLaterErelQ6 = {}
    perLaterErelBF = {}
    for key in perLayerBitSize[model]:
        if key.find('relu')!=-1 or key.find('softmax')!=-1 or key.find('bn')!=-1:
            continue
        perLayerBrelQ6[key] = perLayerBitSize[model][key]/perLayerBitSize['Q6'][key]
        perLayerBrelBF[key] = perLayerBitSize[model][key]/perLayerBitSize['BF'][key]
    for key in perLayerEnergy[model]:
        if key.find('relu')!=-1 or key.find('softmax')!=-1 or key.find('bn')!=-1:
            continue
        perLaterErelQ6[key] = perLayerEnergy[model][key]/perLayerEnergy['Q6'][key]
        perLaterErelBF[key] = perLayerEnergy[model][key]/perLayerEnergy['BF'][key]   
    print('bits/Q6 = {}'.format(perLayerBrelQ6))
    print('bits/BF = {}'.format(perLayerBrelBF))
    print('ener/Q6 = {}'.format(perLaterErelQ6))
    print('ener/BF = {}'.format(perLaterErelBF))
    #pprint.pprint('{} {:.0f}     {:.0f}       {:.2f}     {:.2f}     {:.2f}     {:.2f}'.format('{} QTools'.format(model), total_bitsize[model], total_energy[model], total_bitsize[model]/total_bitsize['BF'], total_bitsize[model]/total_bitsize['Q6'], total_energy[model]/total_energy['BF'], total_energy[model]/total_energy['Q6']))
    #pprint.pprint('{} {}         {}          {}          {}      {:.2f}   {:.2f}'.format('{} Vivado'.format(model), '-', '-', '-', '-', vivadoEBF[i], vivadoEQ6[i])) 
    #pprint.pprint('')


 For model BF
bits/Q6 = {'fc1': 2.5, 'fc2': 2.5, 'fc3': 2.5, 'output': 2.5}
bits/BF = {'fc1': 1.0, 'fc2': 1.0, 'fc3': 1.0, 'output': 1.0}
ener/Q6 = {'fc1': 2.2148814036607822, 'fc2': 4.641891891891892, 'fc3': 4.708715596330276, 'output': 4.70862385321101}
ener/BF = {'fc1': 1.0, 'fc2': 1.0, 'fc3': 1.0, 'output': 1.0}

 For model Q6
bits/Q6 = {'fc1': 1.0, 'fc2': 1.0, 'fc3': 1.0, 'output': 1.0}
bits/BF = {'fc1': 0.4, 'fc2': 0.4, 'fc3': 0.4, 'output': 0.4}
ener/Q6 = {'fc1': 1.0, 'fc2': 1.0, 'fc3': 1.0, 'output': 1.0}
ener/BF = {'fc1': 0.4514914425427873, 'fc2': 0.21542940320232898, 'fc3': 0.21237213833414514, 'output': 0.21237627620606345}

 For model QE
bits/Q6 = {'fc1': 0.3137254901960784, 'fc2': 0.08205128205128205, 'fc3': 0.08080808080808081}
bits/BF = {'fc1': 0.12549019607843137, 'fc2': 0.03282051282051282, 'fc3': 0.03232323232323232}
ener/Q6 = {'fc1': 0.3642640528538936, 'fc2': 0.06036036036036037, 'fc3': 0.07597477064220183}
ener/BF = {'fc1': 0.16446210268948655, 'fc2': 0.013003396